# Task 2: Sample Bengali dataset and do preprocessing

In [1]:
# Imports
import re
import string
import json
from datetime import datetime
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import Module
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from nltk.corpus import stopwords

device = 'cpu'

import random

random.seed(26)
np.random.seed(62)
torch.manual_seed(2021)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load Bengali datasets

In [2]:
# load sample Bengali datasets
bengali_train_df = pd.read_csv('../../Task_2/hindi_bengali/save/bengali_hatespeech_sample_train.csv')
bengali_test_df = pd.read_csv('../../Task_2/hindi_bengali/save/bengali_hatespeech_sample_test.csv')

# Preprocessing

In [3]:
train_sentences = bengali_train_df['sentence']
test_sentences = bengali_test_df['sentence']

In [4]:
# remove user taggings
user_tag_pattern = re.compile(r'\@\w*')
def remove_tagging(sentence):
    return re.sub(user_tag_pattern, ' ', sentence)

# remove punctuations and urls
http_re = re.compile('http://[^ ]*')
https_re = re.compile('https://[^ ]*')
punctuation = string.punctuation[:2] + string.punctuation[3:]
translator = str.maketrans(punctuation, ' '*len(punctuation))
def remove_punc_and_urls(s):
    s = re.sub(http_re, ' ', s)
    s = re.sub(https_re, ' ', s)
    s = s.translate(translator)
    return s

# substitute numbers
#   when there is a number in the string:
#   if that number is 0 or 1 or 2, then there is no change.
#   else, that number is substituted by a word describing how many digits it has.
def substitute_number(x):
    x = x.group(0)
    if x in {'0', '1', '2'}:
        return x
    return '{}_digits_number'.format(len(x))

# stopwords BENGALI (source: https://www.ranks.nl/stopwords/bengali)
stopwords = ['অবশ্য', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অথবা', 'অথচ', 'অর্থাত', 'অন্য', 'আজ', 'আছে', 'আপনার', 
             'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি', 'আরও', 'আর', 'আগে', 'আগেই', 'আই', 
             'অতএব', 'আগামী', 'অবধি', 'অনুযায়ী', 'আদ্যভাগে', 'এই', 'একই', 'একে', 'একটি', 'এখন', 'এখনও', 'এখানে', 
             'এখানেই', 'এটি', 'এটা', 'এটাই', 'এতটাই', 'এবং', 'একবার', 'এবার', 'এদের', 'এঁদের', 'এমন', 'এমনকী', 'এল', 
             'এর', 'এরা', 'এঁরা', 'এস', 'এত', 'এতে', 'এসে', 'একে', 'এ', 'ঐ', ' ই', 'ইহা', 'ইত্যাদি', 'উনি', 'উপর', 
             'উপরে', 'উচিত', 'ও', 'ওই', 'ওর', 'ওরা', 'ওঁর', 'ওঁরা', 'ওকে', 'ওদের', 'ওঁদের', 'ওখানে', 'কত', 'কবে', 
             'করতে', 'কয়েক', 'কয়েকটি', 'করবে', 'করলেন', 'করার', 'কারও', 'করা', 'করি', 'করিয়ে', 'করার', 'করাই', 
             'করলে', 'করলেন', 'করিতে', 'করিয়া', 'করেছিলেন', 'করছে', 'করছেন', 'করেছেন', 'করেছে', 'করেন', 'করবেন', 
             'করায়', 'করে', 'করেই', 'কাছ', 'কাছে', 'কাজে', 'কারণ', 'কিছু', 'কিছুই', 'কিন্তু', 'কিংবা', 'কি', 'কী', 'কেউ', 
             'কেউই', 'কাউকে', 'কেন', 'কে', 'কোনও', 'কোনো', 'কোন', 'কখনও', 'ক্ষেত্রে', 'খুব	গুলি', 'গিয়ে', 'গিয়েছে', 
             'গেছে', 'গেল', 'গেলে', 'গোটা', 'চলে', 'ছাড়া', 'ছাড়াও', 'ছিলেন', 'ছিল', 'জন্য', 'জানা', 'ঠিক', 'তিনি', 
             'তিনঐ', 'তিনিও', 'তখন', 'তবে', 'তবু', 'তাঁদের', 'তাঁাহারা', 'তাঁরা', 'তাঁর', 'তাঁকে', 'তাই', 'তেমন', 'তাকে', 
             'তাহা', 'তাহাতে', 'তাহার', 'তাদের', 'তারপর', 'তারা', 'তারৈ', 'তার', 'তাহলে', 'তিনি', 'তা', 'তাও', 'তাতে', 
             'তো', 'তত', 'তুমি', 'তোমার', 'তথা', 'থাকে', 'থাকা', 'থাকায়', 'থেকে', 'থেকেও', 'থাকবে', 'থাকেন', 'থাকবেন', 
             'থেকেই', 'দিকে', 'দিতে', 'দিয়ে', 'দিয়েছে', 'দিয়েছেন', 'দিলেন', 'দু', 'দুটি', 'দুটো', 'দেয়', 'দেওয়া', 'দেওয়ার', 
             'দেখা', 'দেখে', 'দেখতে', 'দ্বারা', 'ধরে', 'ধরা', 'নয়', 'নানা', 'না', 'নাকি', 'নাগাদ', 'নিতে', 'নিজে', 'নিজেই', 
             'নিজের', 'নিজেদের', 'নিয়ে', 'নেওয়া', 'নেওয়ার', 'নেই', 'নাই', 'পক্ষে', 'পর্যন্ত', 'পাওয়া', 'পারেন', 'পারি', 'পারে', 
             'পরে', 'পরেই', 'পরেও', 'পর', 'পেয়ে', 'প্রতি', 'প্রভৃতি', 'প্রায়', 'ফের', 'ফলে', 'ফিরে', 'ব্যবহার', 'বলতে', 
             'বললেন', 'বলেছেন', 'বলল', 'বলা', 'বলেন', 'বলে', 'বহু', 'বসে', 'বার', 'বা', 'বিনা', 'বরং', 'বদলে', 'বাদে', 
             'বার', 'বিশেষ', 'বিভিন্ন	বিষয়টি', 'ব্যবহার', 'ব্যাপারে', 'ভাবে', 'ভাবেই', 'মধ্যে', 'মধ্যেই', 'মধ্যেও', 'মধ্যভাগে', 
             'মাধ্যমে', 'মাত্র', 'মতো', 'মতোই', 'মোটেই', 'যখন', 'যদি', 'যদিও', 'যাবে', 'যায়', 'যাকে', 'যাওয়া', 'যাওয়ার', 
             'যত', 'যতটা', 'যা', 'যার', 'যারা', 'যাঁর', 'যাঁরা', 'যাদের', 'যান', 'যাচ্ছে', 'যেতে', 'যাতে', 'যেন', 'যেমন', 
             'যেখানে', 'যিনি', 'যে', 'রেখে', 'রাখা', 'রয়েছে', 'রকম', 'শুধু', 'সঙ্গে', 'সঙ্গেও', 'সমস্ত', 'সব', 'সবার', 'সহ', 
             'সুতরাং', 'সহিত', 'সেই', 'সেটা', 'সেটি', 'সেটাই', 'সেটাও', 'সম্প্রতি', 'সেখান', 'সেখানে', 'সে', 'স্পষ্ট', 'স্বয়ং', 
             'হইতে', 'হইবে', 'হৈলে', 'হইয়া', 'হচ্ছে', 'হত', 'হতে', 'হতেই', 'হবে', 'হবেন', 'হয়েছিল', 'হয়েছে', 'হয়েছেন', 'হয়ে', 
             'হয়নি', 'হয়', 'হয়েই', 'হয়তো', 'হল', 'হলে', 'হলেই', 'হলেও', 'হলো', 'হিসাবে', 'হওয়া', 'হওয়ার', 'হওয়ায়', 'হন', 
             'হোক', 'জন', 'জনকে', 'জনের', 'জানতে', 'জানায়', 'জানিয়ে', 'জানানো', 'জানিয়েছে', 'জন্য', 'জন্যওজে', 'জে', 
             'বেশ', 'দেন', 'তুলে', 'ছিলেন', 'চান', 'চায়', 'চেয়ে', 'মোট', 'যথেষ্ট', 'টি']

In [5]:
def clean_texts(sentences):
    # tags
    sentences = [remove_tagging(sentence) for sentence in sentences]
    
    # lower case
    sentences = [sentence.lower() for sentence in sentences]
    
    # remove punctuations and urls 
    sentences = [remove_punc_and_urls(sentence) for sentence in sentences]
    
    # substitute numbers
    sentences = [re.sub('\\b[0-9]+\\b', substitute_number, sentence) for sentence in sentences]
    
    # remove stopwords
    sentences = [[word for word in sentence.split() if word not in stopwords] for sentence in sentences]
    
    return sentences

In [6]:
# perform cleaning

train_sentences = clean_texts(train_sentences)
train_texts = [' '.join(l) for l in train_sentences]
bengali_train_df['sentence'] = train_texts

test_sentences = clean_texts(test_sentences)
test_texts = [' '.join(l) for l in test_sentences]
bengali_test_df['sentence'] = test_texts

In [7]:
print('train:')
display(bengali_train_df.head())
print('test:')
display(bengali_test_df.head())

train:


,sentence,hate,category
0,সমকামী হুজুর,0,religion
1,ছাএলীগ সালা দের নিসিদ্দ হক,1,politics
2,কাওয়া গদি ছারলে বুজবে জুতা কেমনে খায়,0,politics
3,কাউয়া কাদের বড় মাগীখোর ভিডিও পিক দেখলে বুঝা লু...,1,politics
4,অপু ভালো কথা ছোট করবেনা,0,"Meme, TikTok and others"


test:


,sentence,hate,category
0,মহিলাকে রিমানডে,0,crime
1,তুর রিপাতকে মন চাইছিল ছেড়ে গেলি সাথে মিত্যে অ...,0,crime
2,হুমায়ুন আজাদ এতো বড় ক্রাক মাতাল,0,religion
3,বাংাল দেশের সবচেয়ে সাহসি ইউটুবার👍👍👍,0,politics
4,কাদের ধনের মাথা চোর কাদের একটা ফাটা কেষ্ট কোই ...,1,politics


## Vocab and Word <-> int transformation

In [8]:
train_sentences = [sentence.split() for sentence in bengali_train_df['sentence']]
test_sentences = [sentence.split() for sentence in bengali_test_df['sentence']]

flattened_words = [word for sentence in train_sentences for word in sentence]
V = sorted(list(set(flattened_words)))
vocab_size = len(V)
print(f'vocab_size: {vocab_size}')

word_to_int = {}
int_to_word = {}
for i, word in enumerate(V):
    word_to_int[word] = i
    int_to_word[i] = word

# save dicts for transformation word <-> int
with open('save/bengali_word_to_int_dict.json', 'w') as f:
    json.dump(word_to_int, f)
with open('save/bengali_int_to_word_dict.json', 'w') as f:
    json.dump(int_to_word, f)    

# save word-counter for sampling
word_counter = Counter(flattened_words)
with open('save/bengali_word_counter.json', 'w') as f:
    json.dump(word_counter, f)

vocab_size: 15190


In [9]:
train_sentences = [[word_to_int[word] for word in sentence] for sentence in train_sentences]
test_sentences = [[word_to_int[word] for word in sentence if word in word_to_int] for sentence in test_sentences]

In [10]:
# exclude empty sentences

train_texts = [' '.join([str(v) for v in l]) for l in train_sentences]
bengali_train_df['sentence'] = train_texts
bengali_train_df = bengali_train_df[bengali_train_df['sentence'].str.len() != 0].reset_index(drop=True)

test_texts = [' '.join([str(v) for v in l]) for l in test_sentences]
bengali_test_df['sentence'] = test_texts
bengali_test_df = bengali_test_df[bengali_test_df['sentence'].str.len() != 0].reset_index(drop=True)

## Save preprocessed datasets

In [11]:
display(bengali_train_df.head())
display(bengali_test_df.head())

,sentence,hate,category
0,13549 14750,0,religion
1,4677 13865 6879 7552 14359,1,politics
2,2563 3604 4734 10256 5319 3038 3435,0,politics
3,2556 2633 10699 11348 10994 8322 6775 10290 12...,1,politics
4,388 10938 2116 4863 2323,0,"Meme, TikTok and others"


,sentence,hate,category
0,11279,0,crime
1,6010 11176 4155 4831 3905 13755 469 2339 6616 ...,0,crime
2,14766 798 1815 10699 11394,0,religion
3,6910 13519 13912,0,politics
4,2633 6992 11400 4605 2633 1669 8999 3079 12437...,1,politics


In [12]:
bengali_train_df.to_csv('save/bengali_hatespeech_sample_train_preprocessed.csv', index=False)
bengali_test_df.to_csv('save/bengali_hatespeech_sample_test_preprocessed.csv', index=False)